<a href="https://colab.research.google.com/github/Efradgalio/Employee-Sentiment-Tracker/blob/main/EDA_Empl_Opinion_Tracker_r2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis (EDA)

EDA is done from preprocessing11.csv data, from the existing results obtained several things:


- data consisting of 27013 rows and 16 columns ['Title', 'Place' 'Job_type', 'Department' 'Date', 'Overall_rating', 'work_life_balance','skill_development','salary_and_benefits', 'job_security', 'career_growth', 'works_satisfaction', 'Likes' 'Dislikes', 'likes_cleaned', 'dislikes_ cleaned']
- data that in each attribute is not balanced and there is a lot of null
- because the objective of the Employee opinion tracker is to see sentiment opinion then the data that performs the processing more on Likes_cleaned and Dislike_cleaned
- N Gram and Occurance outcomes of (Likes_clineed & Dislike-cLEaned) can be some topics that can be used as a basis for the next process

Coding process as shown bellow

# Library

In [46]:
# Library
import pandas as pd
import numpy as np
import altair as alt
from collections import Counter
from google.colab import drive
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Data Loading

In [2]:
# Load Data
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/preprocessing11.csv'
data = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# rows & columns
num_row, num_col = data.shape
print('The dataset consist of  {} rows and {} columns.'.format(num_row, num_col))

The dataset consist of  27013 rows and 16 columns.


In [4]:
# Check Atribute
print('Attributes : ', data.columns.values)

Attributes :  ['Title' 'Place' 'Job_type' 'Department' 'Date' 'Overall_rating'
 'work_life_balance' 'skill_development' 'salary_and_benefits'
 'job_security' 'career_growth' 'work_satisfaction' 'Likes' 'Dislikes'
 'Likes_cleaned' 'Dislikes_cleaned']


## Data Information

In [52]:
# data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27013 entries, 0 to 27012
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Title                25932 non-null  object 
 1   Place                24613 non-null  object 
 2   Job_type             11576 non-null  object 
 3   Department           22103 non-null  object 
 4   Date                 25935 non-null  object 
 5   Overall_rating       25918 non-null  float64
 6   work_life_balance    26997 non-null  float64
 7   skill_development    26996 non-null  float64
 8   salary_and_benefits  26967 non-null  float64
 9   job_security         26963 non-null  float64
 10  career_growth        26951 non-null  float64
 11  work_satisfaction    26929 non-null  float64
 12  Likes                23884 non-null  object 
 13  Dislikes             22986 non-null  object 
 14  Likes_cleaned        27013 non-null  object 
 15  Dislikes_cleaned     27013 non-null 

In [10]:
data.head(100)

,Title,Place,Job_type,Department,Date,Overall_rating,work_life_balance,skill_development,salary_and_benefits,job_security,career_growth,work_satisfaction,Likes,Dislikes,Likes_cleaned,Dislikes_cleaned
0,Senior Consultant,Pune,Full Time,General Insurance Department,8-Sep-23,4.0,4.0,3.0,3.0,4.0,4.0,4.0,Deserved candidates are promoted promptly.\nUn...,With designation promotions good salary increm...,deserved candidate promoted promptly unbiased ...,designation promotion good salary increment al...
1,Senior Software Engineer,"Kolkata, West Bengal",Full Time,Software Development Department,7-Sep-23,3.0,4.0,4.0,3.0,4.0,4.0,3.0,You got lot of learning platform and monthly l...,You will get fully not tech project.\nThere is...,got lot learning platform monthly learning pla...,get fully tech project variable pay no apprais...
2,Senior Consultant,"Bengaluru/Bangalore, Karnataka",Full Time,Software Development Department,6-Sep-23,4.0,4.0,5.0,3.0,3.0,4.0,3.0,Based on which Business Unit you are you will ...,Some top level management people bring bad nam...,based business unit get experience company pol...,top level management people bring bad name com...
3,Senior Consultant,"Pune, Maharashtra",Full Time,Banking Operations Department,6-Sep-23,1.0,2.0,4.0,1.0,4.0,1.0,1.0,Some client projects are good as they use the ...,"Culture, micro management, unprofessional beha...",client project good use latest tech work,culture micro management unprofessional behavi...
4,Team Lead,"Noida, Uttar Pradesh",Full Time,IT Infrastructure Services Department,5-Sep-23,4.0,5.0,4.0,4.0,4.0,4.0,3.0,"I have worked in IBM (4 years) ,TCS (1 year) ,...",Medical insurance amount is quite low compare ...,worked ibm year tc year rd company year comple...,medical insurance amount quite low compare tc ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Team Lead,Chennai,Full Time,Operations Department,3-Aug-23,4.0,4.0,5.0,3.0,5.0,4.0,3.0,CG as a company is very good for women profess...,No growth in terms of salary hike when you sta...,capgemini company good woman professional good...,growth term salary hike stay three year everyt...
96,Senior Consultant,"Noida, Uttar Pradesh",Full Time,IT Infrastructure Services Department,2-Aug-23,4.0,4.0,3.0,4.0,3.0,4.0,3.0,1. Usually 1 person remains allocated to one p...,"On your exit, remaining amount of health insur...",usually person remains allocated one project t...,exit remaining amount health insurance premium...
97,Senior Analyst,Pune,Full Time,Software Development Department,2-Aug-23,4.0,2.0,4.0,2.0,4.0,3.0,2.0,I got to work on a better ERP.\nLaptop & asset...,Note: This is only based on the project that I...,got work better erp laptop asset received good...,note based project worked experience had work ...
98,Gcp Data Engineer,Pune,Full Time,Data Science & Machine Learning Department,2-Aug-23,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Nothing,Capsgimini don't have any project. They hire p...,nothing,capsgimini project hire people put bench day d...


## Check Data Duplication & Null

In [7]:
# Check duplicate data
duplicate_counts = {}
for column in data.columns:
    duplicate_counts[column] = data.duplicated(subset=[column]).sum()

for column, count in duplicate_counts.items():
    print(f"Number of duplicate entries in {column}: {count}")

Number of duplicate entries in Title: 22768
Number of duplicate entries in Place: 26182
Number of duplicate entries in Job_type: 27007
Number of duplicate entries in Department: 26391
Number of duplicate entries in Date: 26273
Number of duplicate entries in Overall_rating: 27007
Number of duplicate entries in work_life_balance: 27007
Number of duplicate entries in skill_development: 27007
Number of duplicate entries in salary_and_benefits: 27007
Number of duplicate entries in job_security: 27007
Number of duplicate entries in career_growth: 27007
Number of duplicate entries in work_satisfaction: 27007
Number of duplicate entries in Likes: 9514
Number of duplicate entries in Dislikes: 9464
Number of duplicate entries in Likes_cleaned: 11996
Number of duplicate entries in Dislikes_cleaned: 11963


In [8]:
print(data.isnull().sum())

Title                   1081
Place                   2400
Job_type               15437
Department              4910
Date                    1078
Overall_rating          1095
work_life_balance         16
skill_development         17
salary_and_benefits       46
job_security              50
career_growth             62
work_satisfaction         84
Likes                   3129
Dislikes                4027
Likes_cleaned           3292
Dislikes_cleaned        4300
dtype: int64


# EDA Satisfaction data
Overall_rating', 'work_life_balance', 'skill_development', 'salary_and_benefits', 'job_security', 'career_growth', 'work_satisfaction'

## Graph Satisfaction Data

In [57]:
# Define a function to create a frequency dataframe and a bar chart
def create_chart(data, column):
    frequency = data[column].value_counts()
    frequency_df = frequency.reset_index()


    frequency_df.columns = [column, 'Frequency']
    frequency_df = frequency_df.sort_values(column)

    # Chart
    chart = alt.Chart(frequency_df).mark_bar().encode(
        x=alt.X(f'{column}:N', title=column, sort='x'),  # Sort the bars by the column
        y=alt.Y('Frequency:Q', title='Frequency'),
        color=alt.Color(f'{column}:N', legend=alt.Legend(title=column))  # Use the column for the color encoding
    ).properties(width=600, title=f'{column} Bar Chart')


    text = chart.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text='Frequency:Q'
    )
    return (chart + text)

# Create a bar chart
columns = ['Overall_rating', 'work_life_balance', 'skill_development', 'salary_and_benefits', 'job_security', 'career_growth', 'work_satisfaction']
charts = [create_chart(data, column) for column in columns]
combined_chart = alt.vconcat(*charts).resolve_scale(color='independent')

# Display the chart
combined_chart


alt.VConcatChart(...)

# EDA Likes & Dislikes

## Most Frequent Word Dislikes & Likes

In [11]:
dislikes_cleaned = data['Dislikes_cleaned']
dislikes_cleaned_str = [str(entry) for entry in dislikes_cleaned]

word_freq = Counter(" ".join(dislikes_cleaned_str).split())
top_words = word_freq.most_common(100)

# table header
print("{:<15} {:<10}".format("Word", "Count"))
print("-" * 25)

# Print each word and its count in a row
for word, count in top_words:
  print("{:<15} {:<10}".format(word, count))


Word            Count     
-------------------------
work            4909      
nan             4300      
good            3859      
nothing         2693      
salary          2597      
hike            2366      
company         1980      
project         1811      
management      1523      
appraisal       1475      
employee        1473      
growth          1456      
balance         1435      
life            1394      
capgemini       1307      
culture         1268      
le              1163      
job             1155      
compensation    1144      
manager         902       
working         881       
everything      861       
security        854       
team            814       
politics        802       
promotion       794       
career          788       
policy          783       
like            767       
get             763       
opportunity     761       
dislike         757       
people          699       
increment       654       
low             654       
mu

In [12]:

likes_cleaned = data['Likes_cleaned']
likes_cleaned_str = [str(entry) for entry in likes_cleaned]

word_freq_likes_cleaned = Counter(" ".join(likes_cleaned_str).split())
top_words_likes_cleaned = word_freq_likes_cleaned.most_common(100)

print("{:<15} {:<10}".format("Word", "Count"))
print("-" * 25)

for word, count in top_words_likes_cleaned:
  print("{:<15} {:<10}".format(word, count))


Word            Count     
-------------------------
work            8824      
good            8237      
nan             3293      
company         3161      
culture         3056      
balance         2986      
life            2886      
nothing         2021      
capgemini       1990      
job             1820      
environment     1774      
security        1587      
learning        1460      
project         1442      
working         1336      
team            1231      
opportunity     1201      
management      1179      
salary          1057      
employee        1054      
like            971       
growth          962       
place           890       
great           881       
hike            802       
policy          779       
career          746       
skill           744       
experience      728       
manager         721       
get             661       
people          652       
learn           643       
everything      595       
training        582       
fr

## Triple Gram Dislikes & Likes

In [45]:
# N Gram (Triple Gram)
data['Dislikes_cleaned'] = data['Dislikes_cleaned'].astype(str)
data['Likes_cleaned'] = data['Likes_cleaned'].astype(str)

# function to create N-grams with error handling
def create_ngrams(text_col, n):
  """
  This function takes a DataFrame column containing text data and returns a Counter object with N-gram frequencies.
  It handles cases where values in the column might not be strings.

  Args:
      text_col (str): Name of the column containing text data.
      n (int): The number of words in the N-gram.

  Returns:
      Counter: A Counter object containing N-gram frequencies (or an empty Counter if no text data is found).
  """
  ngrams = Counter()
  for text in data[text_col]:
    if isinstance(text, str):
      words = text.split()
      # Generate N-grams
      for i in range(len(words) - n + 1):
        ngrams[tuple(words[i:i + n])] += 1
  return ngrams

# Specify the N-gram
n = 3

try:
  dislikes_cleaned_ngrams = create_ngrams('Dislikes_cleaned', n)
  likes_cleaned_ngrams = create_ngrams('Likes_cleaned', n)

  print(f"Most frequent Dislikes_cleaned {n}-grams:")
  print(dislikes_cleaned_ngrams.most_common(50))

  print(f"\nMost frequent Likes_cleaned {n}-grams:")
  print(likes_cleaned_ngrams.most_common(50))
except (KeyError, TypeError) as e:

  print(f"Error creating N-grams: {e}")


Most frequent Dislikes_cleaned 3-grams:
[(('work', 'life', 'balance'), 1183), (('life', 'balance', 'good'), 103), (('good', 'work', 'environment'), 103), (('work', 'culture', 'good'), 96), (('good', 'place', 'work'), 77), (('good', 'work', 'life'), 73), (('job', 'security', 'work'), 70), (('good', 'work', 'culture'), 70), (('na', 'na', 'na'), 66), (('career', 'growth', 'slow'), 65), (('good', 'company', 'work'), 55), (('nothing', 'nothing', 'nothing'), 47), (('work', 'environment', 'culture'), 44), (('salary', 'hike', 'le'), 39), (('culture', 'job', 'security'), 37), (('great', 'place', 'work'), 36), (('job', 'security', 'good'), 36), (('culture', 'work', 'life'), 35), (('security', 'work', 'culture'), 35), (('work', 'pressure', 'high'), 33), (('salary', 'decent', 'career'), 32), (('decent', 'career', 'growth'), 32), (('salary', 'hike', 'low'), 31), (('work', 'culture', 'job'), 31), (('salary', 'hike', 'promotion'), 31), (('lot', 'politics', 'management'), 30), (('work', 'culture', 'wo

## LDA (Topic)

In [42]:
# Searching Topic using LDA Dislikes
def preprocess_text(text):
  text = text.lower()
  return text.split()

dislikes_cleaned_text = [preprocess_text(text) for text in data['Dislikes_cleaned']]

# Create a dictionary and corpus for LDA
dictionary = Dictionary(dislikes_cleaned_text)
corpus = [dictionary.doc2bow(text) for text in dislikes_cleaned_text]

# Train the LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

# Print the topics
print("\nLDA Topics for Dislikes_cleaned:")
for topic_id, topic in lda_model.print_topics(num_topics=20, num_words=3):
    print(f"Topic {topic_id}:",''.join(topic))



LDA Topics for Dislikes_cleaned:
Topic 0: 0.105*"increment" + 0.065*"facility" + 0.042*"transport"
Topic 1: 0.248*"hike" + 0.088*"le" + 0.076*"salary"
Topic 2: 0.086*"capgemini" + 0.055*"work" + 0.049*"client"
Topic 3: 0.167*"dislike" + 0.083*"nothing" + 0.066*"thing"
Topic 4: 0.641*"nan" + 0.110*"growth" + 0.055*"career"
Topic 5: 0.089*"opportunity" + 0.073*"learning" + 0.056*"skill"
Topic 6: 0.097*"manager" + 0.065*"people" + 0.056*"team"
Topic 7: 0.168*"like" + 0.089*"work" + 0.069*"pressure"
Topic 8: 0.091*"employee" + 0.042*"training" + 0.030*"capgemini"
Topic 9: 0.194*"compensation" + 0.059*"benefit" + 0.034*"office"
Topic 10: 0.065*"experience" + 0.041*"time" + 0.040*"innovation"
Topic 11: 0.186*"everything" + 0.164*"politics" + 0.061*"bad"
Topic 12: 0.125*"policy" + 0.053*"terrible" + 0.047*"hr"
Topic 13: 0.227*"work" + 0.194*"good" + 0.132*"culture"
Topic 14: 0.115*"project" + 0.068*"fresher" + 0.045*"support"
Topic 15: 0.065*"pay" + 0.024*"per" + 0.024*"employee"
Topic 16: 0

In [44]:
# Searching Topic using LDA Likes
def preprocess_text(text):
  text = text.lower()
  return text.split()

# Prepare data for LDA
likes_cleaned_text = [preprocess_text(text) for text in data['Likes_cleaned']]

# Create a dictionary and corpus for LDA
dictionary = Dictionary(likes_cleaned_text)
corpus = [dictionary.doc2bow(text) for text in likes_cleaned_text]

# Train the LDA model (adjust num_topics as needed)
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=50)

# Print the topics
print("\nLDA Topics for Likes_cleaned:")
for topic_id, topic in lda_model.print_topics(num_topics=50, num_words=3):
    print(f"Topic {topic_id}:",''.join(topic))



LDA Topics for Likes_cleaned:
Topic 0: 0.128*"manager" + 0.107*"worklife" + 0.102*"balance"
Topic 1: 0.170*"benefit" + 0.110*"atmosphere" + 0.091*"amazing"
Topic 2: 0.070*"first" + 0.062*"worked" + 0.055*"business"
Topic 3: 0.181*"worst" + 0.103*"hr" + 0.097*"company"
Topic 4: 0.237*"training" + 0.158*"teammate" + 0.150*"fresher"
Topic 5: 0.391*"growth" + 0.270*"career" + 0.068*"travel"
Topic 6: 0.124*"facility" + 0.120*"office" + 0.089*"location"
Topic 7: 0.293*"everything" + 0.240*"politics" + 0.089*"pressure"
Topic 8: 0.402*"environment" + 0.255*"work" + 0.160*"good"
Topic 9: 0.251*"hike" + 0.127*"people" + 0.084*"salary"
Topic 10: 0.185*"increment" + 0.060*"industry" + 0.048*"standard"
Topic 11: 0.308*"work" + 0.294*"life" + 0.264*"balance"
Topic 12: 0.343*"team" + 0.086*"support" + 0.084*"process"
Topic 13: 0.080*"help" + 0.059*"month" + 0.054*"period"
Topic 14: 0.349*"like" + 0.118*"capgemini" + 0.079*"dislike"
Topic 15: 0.160*"best" + 0.135*"year" + 0.073*"company"
Topic 16: 0.

## Co Occurence Likes & Dislikes (Topic)

In [56]:
# Co-occurrence
def calculate_cooccurrence(text_col, window_size=2):
    cooccurrence = Counter()
    for text in data[text_col]:
        words = text.split()
        for i, word in enumerate(words):
            for j in range(1, window_size + 1):
                if i + j < len(words):
                    cooccurrence[(word, words[i + j])] += 1
    return cooccurrence

# N-gram size and window size
n = 3
window_size = 2

try:
    dislikes_cleaned_ngrams = create_ngrams('Dislikes_cleaned', n)
    likes_cleaned_ngrams = create_ngrams('Likes_cleaned', n)
except (KeyError, TypeError) as e:
    print(f"Error creating N-grams: {e}")

# Calculate co-occurrence for Dislikes
dislikes_cooccurrence = calculate_cooccurrence('Dislikes_cleaned', window_size)
likes_cooccurrence = calculate_cooccurrence('Likes_cleaned', window_size)

# Print most frequent co-occurrences (top 20)
print("\nMost Frequent Co-occurrences in Dislikes_cleaned (Window Size:", window_size, ")")
for (word1, word2), count in dislikes_cooccurrence.most_common(20):
    print(f"{word1} -> {word2}: {count}")

print("\nMost Frequent Co-occurrences in Likes_cleaned (Window Size:", window_size, ")")
for (word1, word2), count in likes_cooccurrence.most_common(20):
    print(f"{word1} -> {word2}: {count}")



Most Frequent Co-occurrences in Dislikes_cleaned (Window Size: 2 )
work -> life: 1276
work -> balance: 1212
life -> balance: 1199
job -> security: 812
work -> culture: 810
salary -> hike: 622
good -> work: 567
career -> growth: 550
work -> environment: 333
good -> company: 302
nothing -> dislike: 282
salary -> appraisal: 211
place -> work: 208
work -> pressure: 198
good -> environment: 190
work -> good: 183
worklife -> balance: 180
culture -> good: 170
salary -> increment: 169
company -> work: 169

Most Frequent Co-occurrences in Likes_cleaned (Window Size: 2 )
work -> life: 2700
work -> balance: 2611
life -> balance: 2571
work -> culture: 1962
good -> work: 1595
job -> security: 1492
work -> environment: 909
good -> company: 813
work -> good: 590
good -> environment: 563
place -> work: 543
culture -> good: 509
good -> culture: 436
company -> work: 423
career -> growth: 416
balance -> good: 411
company -> good: 379
good -> good: 371
life -> good: 352
company -> policy: 329
